In [ ]:
!pip install tensorflow
!pip3 install --upgrade tensorflow-gpu --user

#!pip install genism
#!pip install --upgrade gensim # to upgrade version


In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import gensim
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
import gensim.downloader as api
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.text import Tokenizer
text_train_des, text_test_des, y_train_des, y_test_des = train_test_split(df['description'],df['seed_success'], test_size=0.05, random_state=2)

import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MaxAbsScaler, StandardScaler, MinMaxScaler, OneHotEncoder,LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from mlxtend.classifier import StackingClassifier
from mlxtend.feature_selection import ColumnSelector
import tensorflow as tf
import pandas_profiling as pp
import os


In [ ]:
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel

In [ ]:
def one_hot(dataframe, nominal):
    gen_onehot_features = pd.get_dummies(dataframe[nominal])
    result = pd.concat([dataframe, gen_onehot_features],axis=1)
    result.drop([nominal],axis=1,inplace=True)


In [ ]:
df = pd.read_csv('final_seed2.csv', engine='python')

In [ ]:

df.replace(np.nan, 0, inplace=True)
df.replace('NaN', 0, inplace=True)
df.replace(' ', 0, inplace=True)

df = df.fillna(0)

In [ ]:
for index,row in df.iterrows():
      if df.loc[index,'seed_success'] != 0:
            if  df.loc[index,'Interval_month_seed_seriesA'] <= 24:
                   df.loc[index,'seed_success'] =1
         
          
                    
   
   

In [ ]:

df =df[df.announced_year_seed >2007]

In [ ]:

df.replace(np.nan, 0, inplace=True)
df.replace('NaN', 0, inplace=True)
df.replace(' ', 0, inplace=True)

df = df.fillna(0)

In [ ]:
df['crunchbase_id']= pd.to_numeric(df.crunchbase_id, errors='coerce')

In [ ]:
df.groupby(['announced_year_seed']).sum().plot(kind='pie', y='seed_success',autopct='%1.0f%%',legend=False ,startangle=0,shadow =True)

In [ ]:
list= df.columns.tolist()
print(list)


In [ ]:
df[ "description"]=df[ "description"].astype('string')
df[ 'short_description']=df[ 'short_description'].astype('string')
df['techrunch_news_1']=df['techrunch_news_1'].astype('string')
df[ 'techrunch_news_2']=df[ 'techrunch_news_2'].astype('string')

df['tweet_1.1']=df['tweet_1.1'].astype('string')


In [ ]:
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.replace(r"$", "")
    df[text_field] = df[text_field].str.replace(r"%", "")
    df[text_field] = df[text_field].str.replace(r"#", "")
    
      
    df[text_field] = df[text_field].str.lower()
    return df

df = standardize_text(df, "description")
df = standardize_text(df, "short_description")
df = standardize_text(df, "techrunch_news_1")
df = standardize_text(df, "techrunch_news_2")

df = standardize_text(df, "tweet_1.1")


In [ ]:
corr = df.corr()
df.corr()["seed_success"]

In [ ]:
df['num_employees_enum'] = pd.to_numeric(df.num_employees_enum, errors='coerce').fillna(0).astype(np.int64)
df['num_articles'] = pd.to_numeric(df.num_articles, errors='coerce').fillna(0).astype(np.int64)
df['num_event_appearances'] = pd.to_numeric(df.num_event_appearances, errors='coerce').fillna(0).astype(np.int64)
df['num_funding_rounds'] = pd.to_numeric(df.num_funding_rounds, errors='coerce').fillna(0).astype(np.int64)
df['facebook'] = pd.to_numeric(df.facebook, errors='coerce').fillna(0).astype(np.int64)
df['linkedin'] = pd.to_numeric(df.linkedin, errors='coerce').fillna(0).astype(np.int64)
df['twitter'] = pd.to_numeric(df.linkedin, errors='coerce').fillna(0).astype(np.int64)
df['announced_year_seed'] = pd.to_numeric(df.announced_year_seed , errors='coerce').fillna(0).astype(np.int64)
df['money_raised_seed'] = pd.to_numeric(df.money_raised_seed, errors='coerce').fillna(0).astype(np.int64)
df['num_investors_seed'] = pd.to_numeric(df.num_investors_seed, errors='coerce').fillna(0).astype(np.int64)
df['announced_year_a'] = pd.to_numeric(df.announced_year_a, errors='coerce').fillna(0).astype(np.int64)

 

In [ ]:

model = api.load("word2vec-google-news-300", return_path=True)


In [ ]:
word2vec_path = "C:\\Users\\MAHSA/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
#word2vec = model(word2vec_path, binary=True)


In [ ]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, df, generate_missing=False):
    embeddings = df['tokens_des'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return (embeddings)

In [ ]:
class_weight_d = class_weight.compute_class_weight('balanced', np.unique(y_train_des), y_train_des)
class_weight_d = dict(enumerate(class_weight_d))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.fit_on_texts(text_train_des)
X_train_des = tokenizer.texts_to_sequences(text_train_des)
X_test_des = tokenizer.texts_to_sequences(text_test_des)


In [ ]:
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.models import load_model
from keras.layers import InputLayer, Conv1D, Dense, Flatten, MaxPooling1D,Input,Concatenate
from keras import Model

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Adding 1 because of reserved 0 index

maxlen = 100 # or fixed length for improved efficiency
embedding_dim = 300
print('vocabubary size:',vocab_size)
print('max length text:',maxlen)

X_train_des = pad_sequences(X_train_des, padding='post', maxlen=maxlen)
X_test_des = pad_sequences(X_test_des, padding='post', maxlen=maxlen)


vocab_size = len(word_index) + 1
embedding_weights = np.zeros((len(word_index)+1, embedding_dim))
for word,index in word_index.items():
         embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(embedding_dim)
print(embedding_weights.shape)
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_weights, axis=1))
print(nonzero_elements / vocab_size) 

# Deep CNN
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen,weights=[embedding_weights],trainable=False))
model.add(Conv1D(128, 7, activation='relu',padding='same'))
model.add(MaxPooling1D())
model.add(Conv1D(256, 5, activation='relu',padding='same'))
model.add(MaxPooling1D())
model.add(Conv1D(512, 3, activation='relu',padding='same'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

# Kim Yoon CNN
model.save('model.h5')
model_final = load_model('model.h5')
sequence_input = Input(shape=(maxlen,), dtype='int32')

embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            weights=[embedding_weights],
                            input_length=maxlen,
                            trainable=False)

embedded_sequences = embedding_layer(sequence_input)
convs = []
filter_sizes = [3,5,7]

for fsz in filter_sizes:
    x = Conv1D(128, fsz, activation='relu',padding='same')(embedded_sequences)
    x = MaxPooling1D()(x)
    convs.append(x)
    
x = Concatenate(axis=-1)(convs)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(sequence_input, output)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

# Fit model
history = model.fit(X_train_des, y_train_des,
                    class_weight = class_weights_d,
                    epochs=3,
                    verbose=True,
                    validation_data=(X_test_des, y_test_des),
                    batch_size=50)
loss, accuracy = model.evaluate(X_train_des, y_train_des, verbose=True)

#model.history
y = df[ 'seed_success']  
x=df.drop( 'seed_success', axis=1)
x_train['cnn_des']=model.predict(X_train_des)
x_test['cnn_des']= model.predict(X_test_des)

In [ ]:
class_weightsd = class_weight.compute_class_weight('balanced', np.unique(y_train_sdes), y_train_sdes)
class_weights_sd = dict(enumerate(class_weightsd))

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer.fit_on_texts(text_train_sdes)
X_train_sdes = tokenizer.texts_to_sequences(text_train_sdes)
X_test_sdes = tokenizer.texts_to_sequences(text_test_sdes)